In [ ]:
import torch
import torchio as tio
import SimpleITK as sitk
import matplotlib.pyplot as plt
import os
import numpy as np
from scipy.ndimage import zoom

def preview_brats_extraction(t1_path, seg_path, percentages=[0.4, 0.6], save_dir='brats_preview', target_size=(96,96)):
    """
    Preview how tumor extraction will work at the exact same relative positions as IXI slices.
    """
    print('='*10)
    print('PREVIEWING BRATS EXTRACTION')
    print(f'Looking at slices at {percentages} through volume')
    print('='*10)
    
    # Load the raw images first to check dimensions
    t1_img = sitk.ReadImage(t1_path, sitk.sitkFloat32)
    seg_img = sitk.ReadImage(seg_path, sitk.sitkFloat32)
    
    # Convert to numpy arrays with correct orientation
    t1_data = sitk.GetArrayFromImage(t1_img)  # This gives us [Z,Y,X]
    seg_data = sitk.GetArrayFromImage(seg_img)
    
    print(f"Full volume shape: {t1_data.shape}")
    
    # Check unique values in the segmentation slice to identify the background value
    unique_values = np.unique(seg_data)
    print(f"Unique values in segmentation data: {unique_values}")
    # Calculate slice indices
    total_slices = t1_data.shape[0]  # Z dimension
    slice_indices = [int(total_slices * p) for p in percentages]
    
    print(f'For volume of size {total_slices}, selected slices: {slice_indices}')
    
    # Create save directory
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    
    # Extract and visualize the slices
    for i, slice_idx in enumerate(slice_indices):
        # Get the slice data
        t1_slice = t1_data[slice_idx]  # This gives us a Y,X slice
        seg_slice = seg_data[slice_idx]
        
        # Calculate zoom factors
        zoom_y = target_size[0] / t1_slice.shape[0]
        zoom_x = target_size[1] / t1_slice.shape[1]
        
        # Resize both slices to target size
        t1_slice_resized = zoom(t1_slice, (zoom_y, zoom_x), order=1)  # order=1 for linear interpolation
        seg_slice_resized = zoom(seg_slice, (zoom_y, zoom_x), order=0)  # order=0 for nearest neighbor
        
        print(f"\nOriginal slice dimensions: {t1_slice.shape}")
        print(f"Resized dimensions: {t1_slice_resized.shape}")
        
        # Create binary tumor mask (BraTS labels are 1,2,4)
        tumor_mask = (seg_slice_resized > 0).astype(float)
        tumor_only = t1_slice_resized * tumor_mask
        
        # Extract just the tumor region
        if np.any(tumor_mask):
            # Get the bounding box of the tumor
            y_indices, x_indices = np.where(tumor_mask > 0)
            y_min, y_max = np.min(y_indices), np.max(y_indices)
            x_min, x_max = np.min(x_indices), np.max(x_indices)
            
            # Add some padding
            padding = 5
            y_min = max(0, y_min - padding)
            y_max = min(tumor_mask.shape[0], y_max + padding)
            x_min = max(0, x_min - padding)
            x_max = min(tumor_mask.shape[1], x_max + padding)
            
            # Extract the tumor region
            isolated_tumor = t1_slice_resized[y_min:y_max, x_min:x_max]
            isolated_mask = tumor_mask[y_min:y_max, x_min:x_max]
            isolated_view = isolated_tumor * isolated_mask
            
            print(f"Isolated tumor region dimensions: {isolated_view.shape}")
        else:
            isolated_view = np.zeros((50, 50))
        
        # Calculate tumor area
        tumor_area = np.sum(tumor_mask)
        
        # Create visualization
        fig, ax = plt.subplots(2, 3, figsize=(18, 10))
        title = (f'Slice {slice_idx} ({percentages[i]*100}% through volume)\n'
                f'Tumor Area: {tumor_area} pixels\n'
                f'Slice dimensions: {t1_slice_resized.shape} (resized from {t1_slice.shape})')
        fig.suptitle(title)
        
        # Original T1 (resized)
        ax[0,0].imshow(t1_slice_resized, 'gray')
        ax[0,0].set_title('Full Slice (96x96)')
        
        # Segmentation mask
        ax[0,1].imshow(tumor_mask, 'gray')
        ax[0,1].set_title('Tumor Mask (96x96)')
        # Print the unique values in the resized tumor mask
        unique_tumor_mask_values = np.unique(tumor_mask)
        print(f"Unique values in resized tumor mask: {unique_tumor_mask_values}")
        
        # Extracted tumor (full slice)
        ax[0,2].imshow(tumor_only, 'gray')
        ax[0,2].set_title('Extracted Tumor (96x96)')
        
        # Original image before resizing
        ax[1,0].imshow(t1_slice, 'gray')
        ax[1,0].set_title(f'Original Slice ({t1_slice.shape})')
        
        # Isolated tumor view
        ax[1,1].imshow(isolated_view, 'gray')
        ax[1,1].set_title(f'Isolated Tumor Region\n{isolated_view.shape}')
        
        # Overlay on resized
        ax[1,2].imshow(t1_slice_resized, 'gray')
        ax[1,2].imshow(tumor_mask, 'jet', alpha=0.3)
        ax[1,2].set_title('Overlay (96x96)')
        
        # Remove ticks and frames
        for axes_row in ax:
            for axes in axes_row:
                axes.set_xticks([])
                axes.set_yticks([])
                for spine in axes.spines.values():
                    spine.set_visible(False)
        
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, f'brats_preview_slice_{slice_idx}.png'), 
                   bbox_inches='tight', dpi=300)
        plt.close()
    
    print('='*10)
    print(f"Preview images saved to {save_dir}")
    print(f"Slices examined: {slice_indices}")
    print('='*10)
    
    return slice_indices

def sitk_reader(path):
    """
    SITK reader from your pipeline
    """
    image_nii = sitk.ReadImage(str(path), sitk.sitkFloat32)
    if not 'mask' in str(path) and not 'seg' in str(path):
        image_nii = sitk.CurvatureFlow(image1=image_nii, timeStep=0.125, numberOfIterations=3)
    vol = sitk.GetArrayFromImage(image_nii).transpose(2,1,0)
    return vol, None

In [24]:
import os

def process_directory(directory, percentages=[0.4, 0.6], save_dir='brats_preview_resized', target_size=(96, 96)):
    """
    Loop through all files in the given directory and process each .nii.gz file for tumor extraction previews.
    
    Parameters:
    - directory: Path to the directory containing the MRI files.
    - percentages: List of percentages for slice positions (default [0.4, 0.6]).
    - save_dir: Directory to save the preview images (default 'brats_preview_resized').
    - target_size: Desired size for resizing the slices (default (96, 96)).
    """
    print(f"Processing directory: {directory}")
    
    # Create save directory if it doesn't exist
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    
    # Loop through all files in the directory
    for filename in os.listdir(directory):
        # Ignore hidden files (those starting with '.')
        if filename.startswith('.'):
            continue
        
        # Check if the file is a segmentation file (you can adjust this check as needed)
        if 'seg' in filename:
            # Extract the numeric part of the filename (e.g., 01647 from BraTS2021_01647_seg.nii.gz)
            base_name = filename.split('_')[1]  # This gets '01647' from 'BraTS2021_01647_seg.nii.gz'
            
            # Construct full path for T1 and segmentation images
            t1_file = os.path.join(directory.replace('seg', 't1'), filename.replace('_seg', '_t1'))  # Assuming seg and t1 files are paired
            seg_file = os.path.join(directory, filename)
            
            # Check if both T1 and segmentation files exist
            if os.path.exists(t1_file) and os.path.exists(seg_file):
                print(f"Processing: {filename}")
                # Update the save path with the base name (image_01647)
                updated_save_dir = os.path.join(save_dir, f"image_{base_name}")
                preview_brats_extraction(t1_file, seg_file, percentages=percentages, save_dir=updated_save_dir, target_size=target_size)
            else:
                print(f"Skipping: {filename} (Missing corresponding T1 or segmentation file)")
    
    print(f"Finished processing directory {directory}. Preview images saved to {save_dir}")

# Example usage
directory_path = "/home/rd81/projects/data-complete/Data/Test/Brats21/seg"
process_directory(directory_path)


Processing directory: /home/rd81/projects/data-complete/Data/Test/Brats21/seg
Processing: BraTS2021_01228_seg.nii.gz
PREVIEWING BRATS EXTRACTION
Looking at slices at [0.4, 0.6] through volume
Full volume shape: (136, 173, 126)
For volume of size 136, selected slices: [54, 81]

Original slice dimensions: (173, 126)
Resized dimensions: (96, 96)
Isolated tumor region dimensions: (53, 61)

Original slice dimensions: (173, 126)
Resized dimensions: (96, 96)
Isolated tumor region dimensions: (67, 58)
Preview images saved to brats_preview_resized/image_01228
Slices examined: [54, 81]
Processing: BraTS2021_01158_seg.nii.gz
PREVIEWING BRATS EXTRACTION
Looking at slices at [0.4, 0.6] through volume
Full volume shape: (133, 174, 135)
For volume of size 133, selected slices: [53, 79]

Original slice dimensions: (174, 135)
Resized dimensions: (96, 96)
Isolated tumor region dimensions: (37, 29)

Original slice dimensions: (174, 135)
Resized dimensions: (96, 96)
Isolated tumor region dimensions: (49, 

In [28]:
t1_path = "/home/rd81/projects/data-complete/Data/Test/Brats21/t1/BraTS2021_01022_t1.nii.gz"
seg_path = "/home/rd81/projects/data-complete/Data/Test/Brats21/seg/BraTS2021_01022_seg.nii.gz"
preview_brats_extraction(t1_path, seg_path, percentages=[0.4, 0.6], save_dir='brats_preview_resized')

PREVIEWING BRATS EXTRACTION
Looking at slices at [0.4, 0.6] through volume
Full volume shape: (140, 176, 140)
Unique values in segmentation data: [0.0000000e+00 1.1872384e-05 1.7545904e-05 ... 3.9999571e+00 3.9999797e+00
 4.0000000e+00]
For volume of size 140, selected slices: [56, 84]

Original slice dimensions: (176, 140)
Resized dimensions: (96, 96)
Isolated tumor region dimensions: (56, 46)
Unique values in resized tumor mask: [0.00000000e+00 8.58366722e-04 2.39595375e-03 3.33864195e-03
 4.45411820e-03 4.60543297e-03 4.63977456e-03 7.07603060e-03
 8.98755249e-03 1.06759816e-02 1.73492432e-02 1.80459805e-02
 1.80861745e-02 1.88325662e-02 2.12317836e-02 2.61415802e-02
 3.04120053e-02 3.32999602e-02 4.01801243e-02 4.37123328e-02
 4.37639840e-02 5.40811047e-02 7.07360655e-02 7.24538565e-02
 9.40530747e-02 9.86736789e-02 9.98204276e-02 1.02555014e-01
 1.16018780e-01 1.24645136e-01 1.28961712e-01 1.39903024e-01
 1.41608998e-01 1.42815247e-01 1.46656096e-01 1.51715949e-01
 1.71752259e-01 

[56, 84]

In [30]:
import os
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

def preview_brats_extraction(t1_path, seg_path, percentages=[0.4, 0.6], save_dir='brats_preview', target_size=(96, 96)):
    """
    Preview how tumor extraction will work at the exact same relative positions as IXI slices.
    """
    print('='*10)
    print('PREVIEWING BRATS EXTRACTION')
    print(f'Looking at slices at {percentages} through volume')
    print('='*10)
    
    # Load the raw images first to check dimensions
    t1_img = sitk.ReadImage(t1_path, sitk.sitkFloat32)
    seg_img = sitk.ReadImage(seg_path, sitk.sitkFloat32)
    
    # Convert to numpy arrays with correct orientation
    t1_data = sitk.GetArrayFromImage(t1_img)  # This gives us [Z,Y,X]
    seg_data = sitk.GetArrayFromImage(seg_img)
    
    print(f"Full volume shape: {t1_data.shape}")
    
    # Calculate slice indices
    total_slices = t1_data.shape[0]  # Z dimension
    slice_indices = [int(total_slices * p) for p in percentages]
    
    print(f'For volume of size {total_slices}, selected slices: {slice_indices}')
    
    # Create save directory
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    
    # Extract and save the slices
    for i, slice_idx in enumerate(slice_indices):
        # Get the slice data
        t1_slice = t1_data[slice_idx]  # This gives us a Y,X slice
        seg_slice = seg_data[slice_idx]
        
        # Calculate zoom factors
        zoom_y = target_size[0] / t1_slice.shape[0]
        zoom_x = target_size[1] / t1_slice.shape[1]
        
        # Resize both slices to target size
        t1_slice_resized = zoom(t1_slice, (zoom_y, zoom_x), order=1)  # order=1 for linear interpolation
        seg_slice_resized = zoom(seg_slice, (zoom_y, zoom_x), order=0)  # order=0 for nearest neighbor
        
        print(f"\nOriginal slice dimensions: {t1_slice.shape}")
        print(f"Resized dimensions: {t1_slice_resized.shape}")
        
        # Create binary tumor mask (BraTS labels are 1, 2, 4)
        tumor_mask = (seg_slice_resized > 0).astype(float)
        
        # Apply the mask directly to the T1 slice (this will keep only the tumor region)
        tumor_only = t1_slice_resized * tumor_mask
        
        # Create a directory specific to the current image
        base_name = os.path.basename(t1_path).split('_')[1]  # Extract the image base name (e.g., 01647 from BraTS2021_01647_t1.nii.gz)
        specific_save_dir = os.path.join(save_dir, f"image_{base_name}")
        
        if not os.path.isdir(specific_save_dir):
            os.makedirs(specific_save_dir)
        
        # Save the images (each image saved separately)
        tumor_mask_path = os.path.join(specific_save_dir, f"tumor_mask_{slice_idx}.png")
        tumor_only_path = os.path.join(specific_save_dir, f"tumor_only_{slice_idx}.png")
        t1_resized_path = os.path.join(specific_save_dir, f"t1_resized_{slice_idx}.png")
        t1_original_path = os.path.join(specific_save_dir, f"t1_original_{slice_idx}.png")
        
        plt.imsave(tumor_mask_path, tumor_mask, cmap='gray')  # Save the binary tumor mask
        plt.imsave(tumor_only_path, tumor_only, cmap='gray')  # Save the tumor-only image
        plt.imsave(t1_resized_path, t1_slice_resized, cmap='gray')  # Save the resized T1 slice
        plt.imsave(t1_original_path, t1_slice, cmap='gray')  # Save the original T1 slice
        
        print(f"Saved images for slice {slice_idx} in {specific_save_dir}")
    
    print('='*10)
    print(f"Preview images saved to {save_dir}")
    print(f"Slices examined: {slice_indices}")
    print('='*10)
    
    return slice_indices

def process_directory(directory, percentages=[0.4, 0.6], save_dir='brats_tumor_images', target_size=(96, 96)):
    """
    Loop through all files in the given directory and process each .nii.gz file for tumor extraction previews.
    
    Parameters:
    - directory: Path to the directory containing the MRI files.
    - percentages: List of percentages for slice positions (default [0.4, 0.6]).
    - save_dir: Directory to save the preview images (default 'brats_preview_resized').
    - target_size: Desired size for resizing the slices (default (96, 96)).
    """
    print(f"Processing directory: {directory}")
    
    # Create save directory if it doesn't exist
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    
    # Loop through all files in the directory
    for filename in os.listdir(directory):
        # Ignore hidden files (those starting with '.')
        if filename.startswith('.'):
            continue
        
        # Check if the file is a segmentation file (you can adjust this check as needed)
        if 'seg' in filename:
            # Extract the numeric part of the filename (e.g., 01647 from BraTS2021_01647_seg.nii.gz)
            base_name = filename.split('_')[1]  # This gets '01647' from 'BraTS2021_01647_seg.nii.gz'
            
            # Construct full path for T1 and segmentation images
            t1_file = os.path.join(directory.replace('seg', 't1'), filename.replace('_seg', '_t1'))  # Assuming seg and t1 files are paired
            seg_file = os.path.join(directory, filename)
            
            # Check if both T1 and segmentation files exist
            if os.path.exists(t1_file) and os.path.exists(seg_file):
                print(f"Processing: {filename}")
                # Update the save path with the base name (image_01647)
                updated_save_dir = os.path.join(save_dir, f"image_{base_name}")
                preview_brats_extraction(t1_file, seg_file, percentages=percentages, save_dir=updated_save_dir, target_size=target_size)
            else:
                print(f"Skipping: {filename} (Missing corresponding T1 or segmentation file)")
    
    print(f"Finished processing directory {directory}. Preview images saved to {save_dir}")

# Example usage
directory_path = "/home/rd81/projects/data-complete/chosen_tumours/Brats21/seg"
process_directory(directory_path)


Processing directory: /home/rd81/projects/data-complete/chosen_tumours/Brats21/seg
Processing: BraTS2021_01654_seg.nii.gz
PREVIEWING BRATS EXTRACTION
Looking at slices at [0.4, 0.6] through volume
Full volume shape: (132, 174, 136)
For volume of size 132, selected slices: [52, 79]

Original slice dimensions: (174, 136)
Resized dimensions: (96, 96)
Saved images for slice 52 in brats_tumor_images/image_01654/image_01654

Original slice dimensions: (174, 136)
Resized dimensions: (96, 96)
Saved images for slice 79 in brats_tumor_images/image_01654/image_01654
Preview images saved to brats_tumor_images/image_01654
Slices examined: [52, 79]
Processing: BraTS2021_00002_seg.nii.gz
PREVIEWING BRATS EXTRACTION
Looking at slices at [0.4, 0.6] through volume
Full volume shape: (143, 175, 137)
For volume of size 143, selected slices: [57, 85]

Original slice dimensions: (175, 137)
Resized dimensions: (96, 96)
Saved images for slice 57 in brats_tumor_images/image_00002/image_00002

Original slice di